In [ ]:
import requests, os

HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY")

url = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-3-medium-diffusers"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

payload = {
    "inputs": "A beautiful sunset",
    "parameters": {
        "width": 1024,
        "height": 1024,
        "num_inference_steps": 20,
        "guidance_scale": 7.5,
        "negative_prompt": "blurry, low quality, distorted, ugly, bad anatomy"
    }
}

res = requests.post(url, headers=headers, json=payload)

if res.status_code == 200:
    with open("sunset.png", "wb") as f:
        f.write(res.content)
    print("Image saved as sunset.png")
else:
    print(f"Error {res.status_code}: {res.text}")


In [ ]:
import replicate
import requests
from io import BytesIO
from PIL import Image
import os

client = replicate.Client(api_token=os.environ["REPLICATE_API_KEY"])

def generate_and_show_image():
    try:
        # Fix: Use the correct model name and approach
        # The model name format should be "owner/model-name"
        output = client.run(
            "stability-ai/stable-diffusion-3-medium-diffusers",
            input={
                "prompt": "A beautiful sunset over mountains",
                "width": 512,
                "height": 512,
                "num_inference_steps": 20,
                "guidance_scale": 7.5,
                "negative_prompt": "blurry, low quality, distorted, ugly, bad anatomy"
            }
        )
        
        print("Generated image URLs:", output)
        
        # Handle different output formats
        if isinstance(output, list):
            image_url = output[0]
        else:
            image_url = output
        
        # Download the image from the URL
        response = requests.get(image_url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        image = Image.open(BytesIO(response.content))
        
        # Display image in notebook (if running in Jupyter)
        try:
            from IPython.display import display
            display(image)
        except ImportError:
            print("IPython not available, showing image using PIL")
            image.show()
        
        # Optionally save locally
        image.save("generated_sunset.png")
        print("Image saved as generated_sunset.png")
        
    except Exception as e:
        print(f"Error generating image: {e}")
        print("Make sure your REPLICATE_API_KEY environment variable is set correctly")

generate_and_show_image()

In [ ]:
# Alternative function with a different stable diffusion model
def generate_with_alternative_model():
    try:
        # Using a more commonly available model
        output = client.run(
            "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
            input={
                "prompt": "A beautiful sunset over mountains, highly detailed, photorealistic",
                "width": 512,
                "height": 512,
                "num_inference_steps": 20,
                "guidance_scale": 7.5,
                "negative_prompt": "blurry, low quality, distorted, ugly, bad anatomy"
            }
        )
        
        print("Generated image URLs:", output)
        
        # Handle the output
        if isinstance(output, list):
            image_url = output[0]
        else:
            image_url = output
        
        # Download and display
        response = requests.get(image_url)
        response.raise_for_status()
        
        image = Image.open(BytesIO(response.content))
        
        try:
            from IPython.display import display
            display(image)
        except ImportError:
            image.show()
        
        image.save("generated_sunset_alt.png")
        print("Image saved as generated_sunset_alt.png")
        
    except Exception as e:
        print(f"Error with alternative model: {e}")

generate_with_alternative_model()

# Starting

In [ ]:
# Cell 1: Setup and imports
import sys
import os
from typing import Dict, List, Any

# Add the parent directory of 'tools' to sys.path
sys.path.append(os.path.abspath(".."))

from workflows.content_generation.static_image_gen import generate_static_images
from workflows.content_generation.static_image_gen import generate_single_static_image
from workflows.content_generation.static_image_gen import create_image_prompt
from workflows.content_generation.static_image_gen import get_type_specific_requirements
from workflows.content_generation.static_image_gen import get_image_style

In [ ]:
# --- RUN ---
content_strategy = {'brand_colors': '#2c3e50, #3498db, #f8f9fa',
 'brand_tone': 'Professional',
 'content_pillars': ['quality', 'benefits', 'sustainability'],
 'content_types': ['ad_copy',
                   'social_captions',
                   'static_images',
                   'ugc_scripts',
                   'email_creative'],
 'key_messages': ['Saves you time and effort', 'Premium quality you can trust'],
 'product_description': 'Premium organic skincare serum with natural '
                        'ingredients that helps improve skin texture and '
                        'reduce signs of aging. Perfect for busy professionals '
                        'who want quality skincare.',
 'target_audience': 'Busy professionals aged 25-45 who value quality and '
                    'natural products'}

generated_images = generate_static_images(content_strategy, brand_colors=content_strategy['brand_colors'], include_product_visuals=True)
for img in generated_images:
    print(img)

In [ ]:

image_types = {"type": "hero_image", "description": "Main campaign hero image"}
prompt = create_image_prompt(content_strategy, image_type=image_types["type"], brand_colors=content_strategy['brand_colors'],description=image_types["description"]) 
print(prompt)

In [ ]:
def get_image_style(image_type: str) -> str:
    """Get style guidance for different image types"""
    
    styles = {
        "hero_image": "cinematic, professional photography, lifestyle",
        "social_post": "trendy, colorful, social media optimized",
        "ad_creative": "persuasive, clean, conversion-focused",
        "product_showcase": "clean, minimalist, product photography"
    }
    
    return styles.get(image_type, "professional, modern, clean")
image_types = {"type": "hero_image", "description": "Main campaign hero image"}
print(get_image_style(image_type=image_types['type']))

In [ ]:
def get_image_dimensions(image_type: str) -> str:
    """Get optimal dimensions for different image types"""
    
    dimensions = {
        "hero_image": "1920x1072",
        "social_post": "1072x1072",
        "ad_creative": "1200x624",
        "product_showcase": "992x992"
    }

    return dimensions.get(image_type, "1072x1072")
print(get_image_dimensions(image_type=image_types['type']))

In [ ]:
from tools.image_gen import generate_image
style = "cinematic, professional photography, lifestyle"
dimensions = "1072x1072"
# Call generate_image
image_result = generate_image(
    prompt=prompt,
    style=style,             # use your manual style here
    dimensions=dimensions    # use your manual dimensions here
)
print(image_result)